In [1]:
import matplotlib.ticker as plticker
import cv2
import pytesseract
from PIL import Image
import PIL
from pdf2image import convert_from_path
import os
import numpy as np
import pandas as pd
from dbr import DynamsoftBarcodeReader
from matplotlib import pyplot as plt
from skimage.measure import compare_ssim
import imutils
import statistics
from scipy import stats
import random
from difflib import SequenceMatcher
from IPython.display import display, Markdown, Latex
import shutil
%matplotlib inline

In [2]:
def convert_pdf_2_image(uploaded_image_path, uploaded_image,img_size):
    project_dir = os.getcwd()
    os.chdir(uploaded_image_path)
    file_name = str(uploaded_image).replace('.pdf','')
    output_file = file_name+'.jpg'
    pages = convert_from_path(uploaded_image, 200,poppler_path='/Users/kunal/Documents/VdartWorking/Poppler/poppler-0.68.0_x86/poppler-0.68.0/bin/')
    for page in pages:
        page.save(output_file, 'JPEG')
        break
    os.chdir(project_dir)
    img = Image.open(output_file)
    img = img.resize(img_size, PIL.Image.ANTIALIAS)
    img.save(output_file)
    return output_file

In [3]:
def checkEqual1(iterator):
    iterator = iter(iterator)
    try:
        first = next(iterator)
    except StopIteration:
        return True
    return all(first == rest for rest in iterator)
def getText(result, partText,text):
    endindex = result + 80
    partialdetected = text[result:endindex]
    endindex = partialdetected.find("\n")
    fulldetected = partialdetected[:endindex]
    startindex = fulldetected.find(partText)#'er'
    barcodeNumberdetectedname = fulldetected[startindex+3:28]
    return barcodeNumberdetectedname
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [4]:
def realfake(testingPNG, testingPDF, realPNG):
    MAINIMAGEFILEPNG = testingPNG
    MAINPDFFILE = testingPDF
    CLEANARRAYFORMAINIMAGEFILEPNG = cv2.imread(MAINIMAGEFILEPNG)
    x11, y11, z11 = CLEANARRAYFORMAINIMAGEFILEPNG.shape
    CLEANARRAYFORMAINIMAGEFILEPNG = cv2.resize(CLEANARRAYFORMAINIMAGEFILEPNG, dsize=(698, 910), interpolation=cv2.INTER_CUBIC)
    COMPAREIMAGEFILEPNG = realPNG
    CLEANARRAYFORCOMPAREIMAGEFILEPNG = cv2.imread(COMPAREIMAGEFILEPNG) 
    x2, y2, z2 = CLEANARRAYFORCOMPAREIMAGEFILEPNG.shape
    CLEANARRAYFORCOMPAREIMAGEFILEPNG = cv2.resize(CLEANARRAYFORCOMPAREIMAGEFILEPNG, dsize=(698, 910), interpolation=cv2.INTER_CUBIC)
    imageCV = CLEANARRAYFORMAINIMAGEFILEPNG
    barcodeArray = []
    dbr = DynamsoftBarcodeReader()
    dbr.initLicense('t0068MgAAACk+RNudXWjI8IqSZ6cPB04DUpOUhqTHb7YTFYTgM+nuTsx92c7RDeCS6o417Oo6MKjRUn0LnCMqfxz8JHqN74w=') # https://www.dynamsoft.com/CustomerPortal/Portal/Triallicense.aspx
    results = dbr.DecodeFile(MAINPDFFILE)
    textResults = results["TextResults"]
    for textResult in textResults:
        #print(textResult["BarcodeText"])
        barcodeArray.append(textResult["BarcodeText"])
    if len(barcodeArray) == 0:
        #print("NO BARCODE DETECTED!!!!")
        barcodeDetected = "N/A"
    elif len(barcodeArray) == 1:
        barcodeDetected = barcodeArray[0]
    elif len(barcodeArray) > 1:
        working = checkEqual1(barcodeArray)
        if working:
            barcodeDetected = barcodeArray[0]
        else:
            barcodeDetected = "N/A"
    OCRPIC = MAINIMAGEFILEPNG
    value=Image.open(OCRPIC)
    pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files (x86)\Tesseract-OCR\tesseract.exe"
    text = pytesseract.image_to_string(value, config='')
    result = text.find('Receipt#') 
    result2pt1  = text.find('Receipt Number ')
    textpt1 = text[result2pt1:len(text)]
    result2 = textpt1.find('Receipt Number ')
    result1Final = getText(result,'pt',text)
    result2Final = getText(result2, 'er',text)
    if result1Final.endswith(";"):
        result1Final = result1Final[:-1]
    if result2Final.endswith(";"):
        result2Final = result2Final[:-1]
    numSpace1 = result1Final.count(" ")
    numSpace2 = result2Final.count(" ")
    if numSpace1 > 1:
        positionofthing = result1Final.find(" ")
        if not positionofthing == -1:
            result1Final = result1Final[0:positionofthing]    
    if numSpace2 > 1:
        positionofthing2 = result2Final.find(" ")
        if not positionofthing2 == -1:
            result2Final = result2Final[0:positionofthing2]
    result1Final = result1Final.strip()
    result2Final = result2Final.strip()
    equal = False
    if result1Final == result2Final:
        if result1Final == barcodeDetected:
            equal = True
        elif result1Final != barcodeDetected:
            equal = False
    if len(result1Final) < 5 and len(result2Final) < 5:
        equal = False
    if len(result1Final) < 5:
        if result2Final == barcodeDetected:
            equal = True
        else:
            equal = False
    if len(result2Final) < 5:
        if result1Final == barcodeDetected:
            equal = True
        else:
            equal = False
    BARCODEISCORRECT = equal
    result = text.find('Valid from') 
    big = text[result:result+35]
    start = big.find("from")
    startstr = big[start+5:start+15]
    end = big.find("to")
    endstr = big[end+3:end+13]
    start = startstr.split("/")
    end = endstr.split("/")
    work = False
    reason = ""
    if ((len(start) == 3) and (len(end) == 3)):    
        if(start[2] > end[2]):
            work = False
            reason = "Year is before the starting one"
        if(start[2] == end[2]):
            if(end[0] < start[0]):
                work = False
                reason = "Month Doesn't Work"
            if(end[0] > start[0]):
                work = True
                reason = "Month Works"
            if(end[0] == start[0]):
                if(end[1] < start[1]):
                    work = False
                    reason = "Day Doesn't Work"
                if(end[1] > start[1]):
                    work = True
                    reason = "Day Works"
                if(end[1] == start[1]):
                    work = False
                    reason = "Date Same"
        if(start[2] < end[2]):
            work = True
            reason = "Year Works"
        if(int(start[1]) > 31 or int(end[1]) > 31 or int(start[0]) > 12 or int(end[0]) > 12):
            work = False
            reason = "Date out of bounds"
    else:
        work = False
        reason = "The array is wrong"
    DATEWORKING = work
    WHYDATEISWORKING = reason



    titleimgCV = CLEANARRAYFORMAINIMAGEFILEPNG[0:50, 10:650]
    titleimgCV2= CLEANARRAYFORCOMPAREIMAGEFILEPNG[0:50, 10:650]
    original = titleimgCV2 #know real
    duplicate = titleimgCV #dont know
    threshold = 50
    TITLEIMAGECOMPARE = False
    if original.shape == duplicate.shape:
        difference = cv2.subtract(original, duplicate)
        b, g, r = cv2.split(difference)
    x,y = b.shape
    size = x*y
    countb = 0
    countg = 0
    countr = 0
    countb11 = 0
    countg11 = 0
    countr11 = 0
    countb22 = 0
    countg22 = 0
    countr22 = 0
    for startthing1 in b:
        for endthomg1 in startthing1:
            if endthomg1 > 80:
                countb11+=1
            if endthomg1 > 50:
                countb+=1
            if endthomg1 > 30:
                countb22+=1
    for startthing2 in g:
        for endthomg2 in startthing2:
            if endthomg2 > 80:
                countg11+=1
            if endthomg2 > 50:
                countg+=1
            if endthomg2 > 30:
                countg22+=1    

    for startthing3 in r:
        for endthomg3 in startthing3:
            if endthomg3 > 80:
                countr11+=1
            if endthomg3 > 50:
                countr+=1
            if endthomg3 > 30:
                countr22+=1   
    PERCENT_BLUE_DIFFERENCE_TITLE = countb/size*100
    PERCENT_GREEN_DIFFERENCE_TITLE = countg/size*100
    PERCENT_RED_DIFFERENCE_TITLE = countr/size*100
    PERCENT_BLUE_DIFFERENCE_TITLE11 = countb11/size*100
    PERCENT_GREEN_DIFFERENCE_TITLE11 = countg11/size*100
    PERCENT_RED_DIFFERENCE_TITLE11 = countr11/size*100
    PERCENT_BLUE_DIFFERENCE_TITLE22 = countb22/size*100
    PERCENT_GREEN_DIFFERENCE_TITLE22 = countg22/size*100
    PERCENT_RED_DIFFERENCE_TITLE22 = countr22/size*100
    
    if PERCENT_BLUE_DIFFERENCE_TITLE < 15 and PERCENT_GREEN_DIFFERENCE_TITLE < 15 and PERCENT_RED_DIFFERENCE_TITLE < 15:
        TITLEIMAGECOMPARE = True
    
    OCRPIC = COMPAREIMAGEFILEPNG
    value=Image.open(OCRPIC)
    text2 = pytesseract.image_to_string(value, config='')
    yes = 0
    no = 0
    totoal = 0
    for i in range(100):
        startingValueofSTring = random.randint(1,len(text2))
        lengthofString = random.randint(3,10)
        small_string = text2[startingValueofSTring:startingValueofSTring+lengthofString]
        has_small_string = False
        if small_string in text:
            has_small_string = True
        if has_small_string:
            yes+=1
        else:
            no+=1
        totoal+=1
    diffsimiliar = yes/totoal
    diffdifferent = no/totoal
    similiar = similar(text, text2)
    if (similiar < 0.35):
        SIMILARTYBETWEENTEXTINDOCUMENT = False
    else:
        SIMILARTYBETWEENTEXTINDOCUMENT = True
    down2 = 239
    up2 = 248 
    array = CLEANARRAYFORMAINIMAGEFILEPNG
    arrayNew = CLEANARRAYFORMAINIMAGEFILEPNG
    w, h, j = array.shape
    array2 = CLEANARRAYFORCOMPAREIMAGEFILEPNG
    arrayNew2 = CLEANARRAYFORCOMPAREIMAGEFILEPNG
    w2, h2, j2 = array2.shape
    falsepixel = 0
    correctpixel = 0
    errorpixel = 0
    totalpixel = 0
    for x2 in range(w2-1):
        for y2 in range(h2-1):
            singlearray2 = array2[x2, y2]
            r2 = singlearray2[0]
            g2 = singlearray2[1]
            b2 = singlearray2[2]
            if (r2 > down2 and r2 < up2 and g2 > down2 and g2 < up2 and b2 > down2 and b2 < up2):
                arrayNew2[x2, y2] = [0, 255, 0]
                singlearray = array[x2,y2]
                r = singlearray[0]
                g = singlearray[1]
                b = singlearray[2]
                if (r == 0 and g == 0 and b == 0):
                    falsepixel+=1
                elif (r == 0 and g == 255 and b == 0):
                    correctpixel +=1
                else:
                    errorpixel += 1
                totalpixel+=1
            else:
                arrayNew2[x2, y2] = [0, 0, 0]
    img2final = Image.fromarray(arrayNew2)
    im1 = img2final.save("REALImageWaterMark.jpg")
    down = 239#238 ----- 239 
    up = 248 # ---------248
    array = CLEANARRAYFORMAINIMAGEFILEPNG
    array2 = CLEANARRAYFORCOMPAREIMAGEFILEPNG
    arrayNew = CLEANARRAYFORMAINIMAGEFILEPNG
    arrayNew2 = CLEANARRAYFORCOMPAREIMAGEFILEPNG
    w,h,c = array.shape
    w2,h2,c2 = array2.shape
    NUMOFPIXELSHOULDTHEREBUTNOT = 0
    NUMOFPIXELNOTBUTTHERE = 0
    NUMOFCORRECTPIXEL = 0
    totalCOUNT = 0
    for x in range(w-1):
        for y in range(h-1):
            singlearray = array[x, y]
            r = singlearray[0]
            g = singlearray[1]
            b = singlearray[2]
            if (r > down and r < up and g > down and g < up and b > down and b < up):
                definitivVALUE = True
                totalCOUNT+=1
                arrayNew[x, y] = [0, 255, 0]
            else:
                definitivVALUE = False
                arrayNew[x, y] = [0, 0, 0]
            singlearray2 = array2[x, y]
            r2 = singlearray2[0]
            g2 = singlearray2[1]
            b2 = singlearray2[2]
            if (r2 > down and r2 < up and g2 > down and g2 < up and b2 > down and b2 < up):
                pixelDONTKNOW = True
            else:
                pixelDONTKNOW = False
            if(definitivVALUE and pixelDONTKNOW):
                #WHEN PIXEL SHOULD BE IN RANGE AND IS IN RANGE
                NUMOFCORRECTPIXEL+=1
            elif(definitivVALUE and not pixelDONTKNOW):
                #WHEN PIXEL SHOULD BE IN RANGE BUT ISNT
                NUMOFPIXELSHOULDTHEREBUTNOT+=1
            elif(not definitivVALUE and definitivVALUE):
                #WHEN PIXEL SHOULDN'T BE IN RANGE BUT IS
                NUMOFPIXELNOTBUTTHERE+=1      
    img = Image.fromarray(arrayNew)
    file1 = 'REALImageWaterMark.jpg'
    row = []
    column = []
    array = cv2.imread(file1)
    x_length = len(array[0])
    y_length = len(array)
    for i in range(x_length):
        row.append(str(i))
    for i in range(y_length):
        column.append(str(i))
    df = pd.DataFrame(index=row, columns=column)
    array = cv2.imread(file1)
    arrayNew = cv2.imread(file1)
    NUMOFPIXELSHOULDTHEREBUTNOTAVERAGE = 0
    NUMOFPIXELNOTBUTTHEREAVERAGE = 0
    NUMOFCORRECTPIXELAVERAGE = 0
    totalCOUNTAVERAGE = 0
    definitivVALUEAVERAGE = False
    distance = 2
    dis = distance
    for thingthing in range(len(array[0])-distance):
        for thingthingYY in range(len(array)-distance):
            center_XXX = thingthing
            center_YYY = thingthingYY
            distance = dis
            partArrDis = [[0 for x in range(distance*2+1)] for y in range(distance*2+1)]
            center_XX = center_XXX
            center_YY = center_YYY
            for uu in range(distance*2+1):
                for rr in range(distance*2+1):
                    a,b,c = array[uu+(thingthingYY-distance*2+1)][rr+(thingthing-distance*2+1)]
                    theSingleArrayABC = [a,b,c]
                    partArrDis[uu][rr] = theSingleArrayABC
            countGreen = 0
            countBlack = 0
            totalCount = 0
            theXofARRRAY = len(partArrDis)
            theYofARRRAY = len(partArrDis[0])
            for aa in range(theXofARRRAY):
                for bb in range(theYofARRRAY):
                    singleARRAYwPART = partArrDis[aa][bb]
                    rr, gg, bb = singleARRAYwPART
                    if rr < 30 and gg < 30 and bb < 30:
                        countBlack += 1
                    else:
                        countGreen += 1    
                    totalCount+=1
            perceBlack = (countBlack/totalCount) * 100
            perceGreen = (countGreen/totalCount) * 100
            x = perceBlack+perceGreen
            IDK_X = center_XXX-distance*2+1
            IDK_Y = center_YYY-distance*2+1
            if (perceBlack > 40):
                arrayNew[IDK_Y][IDK_X]=[0,0,0]
                definitivVALUEAVERAGE = False
            else:
                arrayNew[IDK_Y][IDK_X] = [0,255,0]
                definitivVALUEAVERAGE = True
            singlearray2 = array2[thingthingYY, thingthing]
            r2 = singlearray2[0]
            g2 = singlearray2[1]
            b2 = singlearray2[2]
            if (r2 > down and r2 < up and g2 > down and g2 < up and b2 > down and b2 < up):
                pixelDONTKNOWAVERAGE = True
            else:
                pixelDONTKNOWAVERAGE = False
            if(definitivVALUEAVERAGE and pixelDONTKNOWAVERAGE):
                #WHEN PIXEL SHOULD BE IN RANGE AND IS IN RANGE
                NUMOFCORRECTPIXELAVERAGE+=1
            elif(definitivVALUEAVERAGE and not pixelDONTKNOWAVERAGE):
                #WHEN PIXEL SHOULD BE IN RANGE BUT ISNT
                NUMOFPIXELSHOULDTHEREBUTNOTAVERAGE+=1
            elif(not definitivVALUEAVERAGE and definitivVALUEAVERAGE):
                #WHEN PIXEL SHOULDN'T BE IN RANGE BUT IS
                NUMOFPIXELNOTBUTTHEREAVERAGE+=1     
            totalCOUNTAVERAGE+=1
    img = Image.fromarray(arrayNew)
    im1 = img.save("GroupedPixeledIMAGE.jpg")
    WATERMARKOUTPUTFINAL=False
    WATERMARKOUTPUTave=False
    WATERMARKOUTPUT2=False
    WATERMARKOUTPUTFINAL=False
    
    
    coorect = correctpixel/totalpixel*100
    worng = falsepixel/totalpixel*100
    error = errorpixel
    aveCorrect = NUMOFCORRECTPIXELAVERAGE/totalCOUNTAVERAGE *100
    aveWRONG = NUMOFPIXELSHOULDTHEREBUTNOTAVERAGE/totalCOUNTAVERAGE *100
    aveERROR = NUMOFPIXELNOTBUTTHEREAVERAGE
    correctPT2 = NUMOFCORRECTPIXEL/totalCOUNT *100
    wrongPT2= NUMOFPIXELSHOULDTHEREBUTNOT/totalCOUNT*100
    errorPT2 = NUMOFPIXELNOTBUTTHERE
    
    
    if coorect < 10:
        WATERMARKOUTPUT1 = False
    else:
        WATERMARKOUTPUT1 = True
    if aveCorrect < 10:
        WATERMARKOUTPUTave = False
    else:
        WATERMARKOUTPUTave = True
    if correctPT2 < 10:
        WATERMARKOUTPUT2 = False
    else:
        WATERMARKOUTPUT2 = True
    if WATERMARKOUTPUT1 == True and WATERMARKOUTPUTave == True and WATERMARKOUTPUT2 == True: 
        WATERMARKOUTPUTFINAL = True
    elif WATERMARKOUTPUT1 == True and WATERMARKOUTPUTave == True and WATERMARKOUTPUT2 == False: 
        WATERMARKOUTPUTFINAL = True
    elif WATERMARKOUTPUT1 == False and WATERMARKOUTPUTave == True and WATERMARKOUTPUT2 == True: 
        WATERMARKOUTPUTFINAL = True
    elif WATERMARKOUTPUT1 == True and WATERMARKOUTPUTave == False and WATERMARKOUTPUT2 == True: 
        WATERMARKOUTPUTFINAL = True
    img1 = CLEANARRAYFORMAINIMAGEFILEPNG
    img2 = CLEANARRAYFORCOMPAREIMAGEFILEPNG
    imageA = CLEANARRAYFORMAINIMAGEFILEPNG
    imageB = CLEANARRAYFORCOMPAREIMAGEFILEPNG
    grayA = cv2.cvtColor(imageA, cv2.COLOR_BGR2GRAY)
    grayB = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)
    bigH, bigW,other = imageA.shape
    (score, diff) = compare_ssim(grayA, grayB, full=True)
    diff = (diff * 255).astype('uint8')
    thresh = cv2.threshold(diff, 0, 255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    areaDIFFERENCE = 0
    for c in cnts:
        (x, y, w, h) = cv2.boundingRect(c)
        cv2.rectangle(imageA, (x, y), (x + w, y + h), (0, 0, 255), 2)
        cv2.rectangle(imageB, (x, y), (x + w, y + h), (0, 0, 255), 2)
        areaDIFFERENCE += (w*h)
    perdiff =  ((areaDIFFERENCE/(bigW*bigH))*100)
    
    
    gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    sift = cv2.xfeatures2d.SIFT_create()
    keypoints_1, descriptors_1 = sift.detectAndCompute(img1,None)
    keypoints_2, descriptors_2 = sift.detectAndCompute(img2,None)
    NUMBER_OF_KEYPOINST_IMG1 = len(keypoints_1)
    NUMBER_OF_KEYPOINST_IMG2 = len(keypoints_2)
    img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    keypoints_1, descriptors_1 = sift.detectAndCompute(img1,None)
    keypoints_2, descriptors_2 = sift.detectAndCompute(img2,None)
    bf = cv2.BFMatcher(cv2.NORM_L1, crossCheck=True)
    matches = bf.match(descriptors_1,descriptors_2)
    matches = sorted(matches, key = lambda x:x.distance)
    totalMATCHES = (len(matches))
    if img1 is None or img2 is None:
        exit(0)
    minHessian = 400
    detector = cv2.xfeatures2d_SURF.create(hessianThreshold=minHessian)
    keypoints1, descriptors1 = detector.detectAndCompute(img1, None)
    keypoints2, descriptors2 = detector.detectAndCompute(img2, None)
    matcher = cv2.DescriptorMatcher_create(cv2.DescriptorMatcher_FLANNBASED)
    knn_matches = matcher.knnMatch(descriptors1, descriptors2, 2)
    ratio_thresh1 = 0.6
    good_matches1 = []
    for m,n in knn_matches:
        if m.distance < ratio_thresh1 * n.distance:
            good_matches1.append(m)
    ratio_thresh2 = 0.65
    good_matches2 = []
    for m,n in knn_matches:
        if m.distance < ratio_thresh2 * n.distance:
            good_matches2.append(m) 
    ratio_thresh3 = 0.7
    good_matches3 = []
    for m,n in knn_matches:
        if m.distance < ratio_thresh3 * n.distance:
            good_matches3.append(m)
    NUMEBROFMATCHESRATIOTESTFOR06 = len(good_matches1)
    NUMEBROFMATCHESRATIOTESTFOR065 = len(good_matches2)
    NUMEBROFMATCHESRATIOTESTFOR07 = len(good_matches3)
    img_matches = np.empty((max(img1.shape[0], img2.shape[0]), img1.shape[1]+img2.shape[1], 3), dtype=np.uint8)
    img3 = cv2.drawMatches(img1, keypoints1, img2, keypoints2, good_matches2, img_matches, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    im = Image.fromarray(img3)
    im.save('FINALWORKINGIMAGEPT1.png')
    kp1, des1 = sift.detectAndCompute(img1,None)
    kp2, des2 = sift.detectAndCompute(img2,None)
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1,des2, k=2)
    good1 = []
    for m,n in matches:
        if m.distance < 80:
            good1.append([m])
        if n.distance < 80:
            good1.append([n])
    good2 = []
    for m,n in matches:
        if m.distance < 85:
            good2.append([m])
        if n.distance < 85:
            good2.append([n])
    good3 = []
    for m,n in matches:
        if m.distance < 90:
            good3.append([m])
        if n.distance < 90:
            good3.append([n])         
    NUMEBROFMATCHESDISTANCETESTFOR80 = len(good1)
    NUMEBROFMATCHESDISTANCETESTFOR85 = len(good2)
    NUMEBROFMATCHESDISTANCETESTFOR90 = len(good3)
    img3 = cv2.drawMatchesKnn(img1,kp1,img2,kp2,good2,None,flags =2)
    im = Image.fromarray(img3)
    im.save('FINALWORKINGIMAGEPT2.png')
    if score > 0.25:
        SSIMFINALSCORE = True
    else:
        SSIMFINALSCORE = False
    if perdiff > 0.8:
        PERCENTDIFFERNCE = True
    else:
        PERCENTDIFFERNCE = False
    countREAL = 0
    countFAKE = 0
    if BARCODEISCORRECT:
        countREAL += 1
    else:
        countFAKE += 1
    #------------------------
    if DATEWORKING:
        countREAL += 1
    else:
        countFAKE += 1
    #------------------------
    if TITLEIMAGECOMPARE:
        countREAL += 1
    else:
        countFAKE += 1
    #------------------------
    if SIMILARTYBETWEENTEXTINDOCUMENT:
        countREAL += 1
    else:
        countFAKE += 1
    #------------------------
    if WATERMARKOUTPUTFINAL:
        countREAL += 1
    else:
        countFAKE += 1
    #------------------------
    if SSIMFINALSCORE:
        countREAL += 1
    else:
        countFAKE += 1
    #------------------------
    if PERCENTDIFFERNCE:
        countREAL += 1
    else:
        countFAKE += 1
    documentanswer = False
    if countREAL >= 4:
        documentanswer = True
    if documentanswer:
        display(Markdown('**Document is REAL**'))
    else:
        display(Markdown('**Document is FAKE**'))
    return (MAINIMAGEFILEPNG, MAINPDFFILE, COMPAREIMAGEFILEPNG, 
            result1Final, result2Final, barcodeDetected, 
            DATEWORKING, WHYDATEISWORKING,
            PERCENT_BLUE_DIFFERENCE_TITLE, PERCENT_GREEN_DIFFERENCE_TITLE, PERCENT_RED_DIFFERENCE_TITLE, #50
            PERCENT_BLUE_DIFFERENCE_TITLE11, PERCENT_GREEN_DIFFERENCE_TITLE11, PERCENT_RED_DIFFERENCE_TITLE11, #80
            PERCENT_BLUE_DIFFERENCE_TITLE22, PERCENT_GREEN_DIFFERENCE_TITLE22, PERCENT_RED_DIFFERENCE_TITLE22, #30
            TITLEIMAGECOMPARE,
            similiar, diffsimiliar, diffdifferent,
            coorect, worng, error,
            aveCorrect, aveWRONG, aveERROR,
            correctPT2,wrongPT2, errorPT2,
            WATERMARKOUTPUTFINAL,
            score,
            perdiff, 
            NUMBER_OF_KEYPOINST_IMG1, NUMBER_OF_KEYPOINST_IMG2, # number 1 is unknow image
            totalMATCHES,
            NUMEBROFMATCHESRATIOTESTFOR06, NUMEBROFMATCHESRATIOTESTFOR065, NUMEBROFMATCHESRATIOTESTFOR07,
            NUMEBROFMATCHESDISTANCETESTFOR80, NUMEBROFMATCHESDISTANCETESTFOR85, NUMEBROFMATCHESDISTANCETESTFOR90)

In [14]:
def appendthefiles(a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z,a1,b1,c1,d1,e1,f1,g1,h1,i1,j1,k1,l1,m1,n1,o1,p1):
    aARRAY.append(a)
    bARRAY.append(b)
    cARRAY.append(c)
    dARRAY.append(d)
    eARRAY.append(e)
    fARRAY.append(f)
    gARRAY.append(g)
    hARRAY.append(h)
    iARRAY.append(i)
    jARRAY.append(j)
    kARRAY.append(k)
    lARRAY.append(l)
    mARRAY.append(m)
    nARRAY.append(n)
    oARRAY.append(o)
    pARRAY.append(p)
    qARRAY.append(q)
    rARRAY.append(r)
    sARRAY.append(s)
    tARRAY.append(t)
    uARRAY.append(u)
    vARRAY.append(v)
    wARRAY.append(w)
    xARRAY.append(x)
    yARRAY.append(y)
    zARRAY.append(z)
    a1ARRAY.append(a1)
    b1ARRAY.append(b1)
    c1ARRAY.append(c1)
    d1ARRAY.append(d1)
    e1ARRAY.append(e1)
    f1ARRAY.append(f1)
    g1ARRAY.append(g1)
    h1ARRAY.append(h1)
    i1ARRAY.append(i1)
    j1ARRAY.append(j1)
    k1ARRAY.append(k1)
    l1ARRAY.append(l1)
    m1ARRAY.append(m1)
    n1ARRAY.append(n1)
    o1ARRAY.append(o1)
    p1ARRAY.append(p1)

In [15]:
aARRAY,bARRAY,cARRAY,dARRAY,eARRAY,fARRAY,gARRAY,hARRAY,iARRAY,jARRAY,kARRAY,lARRAY,mARRAY,nARRAY,oARRAY,pARRAY,qARRAY, rARRAY,sARRAY,tARRAY,uARRAY,vARRAY,wARRAY,xARRAY,yARRAY,zARRAY,a1ARRAY,b1ARRAY,c1ARRAY,d1ARRAY,e1ARRAY,f1ARRAY,g1ARRAY,h1ARRAY,i1ARRAY,j1ARRAY,k1ARRAY,l1ARRAY,m1ARRAY,n1ARRAY,o1ARRAY,p1ARRAY = ([] for i in range(42))

In [23]:
%%time
FILEREAL = '/Users/kunal/Documents/VdartWorking/realFake/Document.jpg'
os.chdir('/Users/kunal/Documents/VdartWorking/NEWDOCUMENTS/Working/Cut10/Real/')
count = 0
realDIR = "/Users/kunal/Documents/VdartWorking/NEWDOCUMENTS/Working/Cut10/Real/"
fakeDIR = "/Users/kunal/Documents/VdartWorking/NEWDOCUMENTS/Working/Cut10/Fake/"
for i in os.listdir(realDIR):
    pathName = realDIR + i.replace('.pdf', '')
    os.mkdir(pathName)
    shutil.move(i, pathName)
    finalDir = pathName + "/" + i
    PicForI = i
    finalDirPIC = pathName + "/" + PicForI.replace('.pdf', '.jpg')
    os.chdir(pathName)
    convert_pdf_2_image(pathName, i,(698,910))
    pngFile = finalDirPIC
    print(os.getcwd())
    print(pngFile)
    print(finalDir)
    a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z,a1,b1,c1,d1,e1,f1,g1,h1,i1,j1,k1,l1,m1,n1,o1,p1 = realfake(pngFile, finalDir, FILEREAL)
    appendthefiles(a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z,a1,b1,c1,d1,e1,f1,g1,h1,i1,j1,k1,l1,m1,n1,o1,p1)
    print("DONE")
    os.chdir('/Users/kunal/Documents/VdartWorking/NEWDOCUMENTS/Working/Cut10/Real/')

C:\Users\kunal\Documents\VdartWorking\NEWDOCUMENTS\Working\Cut10\Real\3590_001-manikandan
/Users/kunal/Documents/VdartWorking/NEWDOCUMENTS/Working/Cut10/Real/3590_001-manikandan/3590_001-manikandan.jpg
/Users/kunal/Documents/VdartWorking/NEWDOCUMENTS/Working/Cut10/Real/3590_001-manikandan/3590_001-manikandan.pdf


**Document is FAKE**

DONE
C:\Users\kunal\Documents\VdartWorking\NEWDOCUMENTS\Working\Cut10\Real\Charith_H1B
/Users/kunal/Documents/VdartWorking/NEWDOCUMENTS/Working/Cut10/Real/Charith_H1B/Charith_H1B.jpg
/Users/kunal/Documents/VdartWorking/NEWDOCUMENTS/Working/Cut10/Real/Charith_H1B/Charith_H1B.pdf


**Document is FAKE**

DONE
C:\Users\kunal\Documents\VdartWorking\NEWDOCUMENTS\Working\Cut10\Real\H1B - Udaypal Reddy
/Users/kunal/Documents/VdartWorking/NEWDOCUMENTS/Working/Cut10/Real/H1B - Udaypal Reddy/H1B - Udaypal Reddy.jpg
/Users/kunal/Documents/VdartWorking/NEWDOCUMENTS/Working/Cut10/Real/H1B - Udaypal Reddy/H1B - Udaypal Reddy.pdf


...

DONE
C:\Users\kunal\Documents\VdartWorking\NEWDOCUMENTS\Working\Cut10\Real\Mayuri S. H1B
/Users/kunal/Documents/VdartWorking/NEWDOCUMENTS/Working/Cut10/Real/Mayuri S. H1B/Mayuri S. H1B.jpg
/Users/kunal/Documents/VdartWorking/NEWDOCUMENTS/Working/Cut10/Real/Mayuri S. H1B/Mayuri S. H1B.pdf


**Document is FAKE**

DONE
C:\Users\kunal\Documents\VdartWorking\NEWDOCUMENTS\Working\Cut10\Real\Naresh's H1B copy
/Users/kunal/Documents/VdartWorking/NEWDOCUMENTS/Working/Cut10/Real/Naresh's H1B copy/Naresh's H1B copy.jpg
/Users/kunal/Documents/VdartWorking/NEWDOCUMENTS/Working/Cut10/Real/Naresh's H1B copy/Naresh's H1B copy.pdf


**Document is FAKE**

DONE
Wall time: 20min 42s


In [60]:
aARRAYNEW = []
bARRAYNEW = []
cARRAYNEW = []
for i in aARRAY:
    last_char = i[-30:]
    aARRAYNEW.append(last_char)
for i in bARRAY:
    last_char = i[-30:]
    bARRAYNEW.append(last_char)
for i in cARRAY:
    last_char = i[-12:]
    cARRAYNEW.append(last_char)

In [76]:
data = {"Testing PNG": aARRAYNEW,
        "Testing PDF": bARRAYNEW,
        "Real PNG" : cARRAYNEW,
        "Barcode Decoded(1) TXT" : dARRAY,
        "Barcode Decoded(2) TXT" : eARRAY,
        "Barcode Decoded Barcode" : fARRAY,
        "Date Working(T/F)": gARRAY,
        "Reason for Date" : hARRAY,
        "Title Blue 50" : iARRAY,
        "Title Green 50" : jARRAY,
        "Title Red 50" : kARRAY,
        "Title Blue 80" : lARRAY,
        "Title Green 80" : mARRAY,
        "Title Red 80" : nARRAY,
        "Title Blue 30" : oARRAY,
        "Title Green 30" : pARRAY,
        "Title Red 30" : qARRAY,
        "Final Title (T/F)" :rARRAY,
        "Similarty SequenceMatcher TXT": sARRAY,
        "Similarty Correct TXT": tARRAY,
        "Similarty Incorrect TXT" : uARRAY,
        "WaterMark Correct 1": vARRAY,
        "WaterMark False 1": wARRAY,
        "WaterMark Error 1": xARRAY,
        "WaterMark Ave Correct 2": yARRAY,
        "WaterMark Ave False 2": zARRAY,
        "WaterMark Ave Error 2": a1ARRAY, 
        "WaterMark Correct 3": b1ARRAY,
        "WaterMark False 3": c1ARRAY,
        "WaterMark Error 3": d1ARRAY,
        "WaterMark Final (T/F)": e1ARRAY,
        "SSIM Score": f1ARRAY,
        "Percent Difference": g1ARRAY,
        "Testing KeyPoints": h1ARRAY,
        "Known KeyPoints": i1ARRAY,
        "Total Matches": j1ARRAY,
        "Matches Ratio 0.6" :k1ARRAY,
        "Matches Ratio 0.65": l1ARRAY,
        "Matches Ratio 0.7": m1ARRAY,
        "Matches Distance 80": n1ARRAY,
        "Matches Distance 85": o1ARRAY,
        "Matches Distance 90": p1ARRAY
       }

In [78]:
os.getcwd()

'C:\\Users\\kunal\\Documents\\VdartWorking\\NEWDOCUMENTS\\Working\\Cut10\\Real'

In [79]:
df = pd.DataFrame(data) 
df.to_excel("FINALOUTPUT.xlsx")  
df

,Testing PNG,Testing PDF,Real PNG,Barcode Decoded(1) TXT,Barcode Decoded(2) TXT,Barcode Decoded Barcode,Date Working(T/F),Reason for Date,Title Blue 50,Title Green 50,...,Percent Difference,Testing KeyPoints,Known KeyPoints,Total Matches,Matches Ratio 0.6,Matches Ratio 0.65,Matches Ratio 0.7,Matches Distance 80,Matches Distance 85,Matches Distance 90
0,kandan/3590_001-manikandan.jpg,kandan/3590_001-manikandan.pdf,Document.jpg,,797A,EAC1916852994,False,The array is wrong,33.953125,32.750000,...,102.820618,5558,8530,3757,15,27,54,39,41,43
1,al/Charith_H1B/Charith_H1B.jpg,al/Charith_H1B/Charith_H1B.pdf,Document.jpg,,,EAC1915852025,True,Year Works,26.265625,23.803125,...,102.141755,3031,8564,2478,44,52,63,106,118,126
2,kandan/3590_001-manikandan.jpg,kandan/3590_001-manikandan.pdf,Document.jpg,,797A,EAC1916852994,False,The array is wrong,33.953125,32.750000,...,102.820618,5558,8530,3757,18,33,61,39,41,43
3,al/Charith_H1B/Charith_H1B.jpg,al/Charith_H1B/Charith_H1B.pdf,Document.jpg,,,EAC1915852025,True,Year Works,26.265625,23.803125,...,102.141755,3031,8564,2478,45,55,72,108,120,128
4,Reddy/H1B - Udaypal Reddy.jpg,Reddy/H1B - Udaypal Reddy.pdf,Document.jpg,wacisisssio7,797A,WAC1816451672,False,The array is wrong,24.112500,23.396875,...,99.735193,13117,7939,4802,2,9,40,0,0,0
5,Gautham Chandra Samudrala.jpg,Gautham Chandra Samudrala.pdf,Document.jpg,,,WAC1928250711,True,Year Works,17.853125,17.625000,...,103.236248,6382,8372,3953,24,29,56,19,22,27
6,H1B-SomaSekhar Reddy Aluva.jpg,H1B-SomaSekhar Reddy Aluva.pdf,Document.jpg,,797A,WAC1917852558,True,Year Works,32.815625,31.221875,...,110.264492,11736,8912,5184,3,12,36,2,2,2
7,latha H-1B Approval Notice.jpg,latha H-1B Approval Notice.pdf,Document.jpg,WAci9179s1017,797A,WAC1917951017,False,The array is wrong,16.793750,16.193750,...,101.915205,4217,8543,3025,26,29,52,111,135,152
8,swari Mandha_Approval Copy.jpg,swari Mandha_Approval Copy.pdf,Document.jpg,WAci919251739,797A,N/A,True,Year Works,36.075000,36.043750,...,103.585913,3257,8594,2620,20,31,46,103,111,115
9,0/Real/Ishaq H1B/Ishaq H1B.jpg,0/Real/Ishaq H1B/Ishaq H1B.pdf,Document.jpg,,,WAC1919351391,True,Year Works,27.400000,26.571875,...,113.718316,11500,8937,4904,4,16,49,0,0,1
